# Generate Polymer Statistics

Load polymer configurations from simulation output directory and calculate basic polymer statistics from configurations.

## Import directories

In [ ]:
import importlib
import os
import sys
from io import StringIO

import numpy as np
import pandas as pd

cwd = os.getcwd()
sys.path[8] = sys.path[0]
os.chdir("../..")
print("Current Working Directory: ")
print(os.getcwd())
sys.path.append(os.getcwd())


## Load local modules

In [ ]:
import chromo.util.poly_stat as ps

## List configuration files

In [ ]:
# Specify the simulation ID on which to run the analysis
sim_ID = 56

# Open the output directory
os.chdir(cwd + "/../../")
output_dir = os.getcwd() + "/output/sim_" + str(sim_ID)

# Generate a list of CSV configuration files in the output directory
output_files = os.listdir(output_dir)
output_files = [f for f in output_files if f.endswith(".csv")]
output_files = [f for f in output_files if len(f.split("_")) == 1]

# Sort the CSV configuration files by snapshot number
snapshot = [int(f.split("-")[-1].split(".")[0]) for f in output_files]
output_files = [f for _, f in sorted(zip(snapshot, output_files))]

## Calculate end-to-end distances

In [ ]:
os.chdir(cwd + "/../../chromo/util")

r2 = []
for i, f in enumerate(output_files):
    if (i+1) % 10 == 0:
        print("Snapshot: " + str(i+1) + " of " + str(len(output_files)))
        print("File: " + f)
        print()
    
    poly_stat = ps.PolyStat("output/sim_" + str(sim_ID) + "/" + f)
    r2.append(
        poly_stat.get_avg_r2(sampling_scheme="overlap_slide", bead_separation=10)
    )

## Visualize polymer statistics

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.plot(r2)
plt.xlabel("Snapshot")
plt.ylabel("Mean R2 (10 nucleosomes)")
plt.show()

## Visualize Polymer Configuration

Why do later snapshots of polymer configuration show such large mean squared R2?

In [ ]:
# Specify path to the final snapshot
final_snapshot = output_files[-1]
os.chdir(cwd)
os.chdir("../../output/sim_" + str(sim_ID))

# Load the final snapshot
final_snapshot_data = pd.read_csv(final_snapshot, skiprows=1, index_col=0, usecols=[0,1,2,3])
final_snapshot_data.head()


In [ ]:
x = final_snapshot_data.x.values
y = final_snapshot_data.y.values
z = final_snapshot_data.z.values

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
ax.scatter3D(x, y, z)
ax.set_xlabel('x', fontsize=16)
ax.set_ylabel('y', fontsize=16)
ax.set_zlabel('z', fontsize=16)
plt.show()